####  Chapter 3 "1. 图，张量, 变量及会话" 有更基础的一些描述
推荐使用tf.get_variable(), 因为：

1. 初始化更方便
比如用xavier_initializer:
```
W = tf.get_variable("W", shape=[784, 256],
       initializer=tf.contrib.layers.xavier_initializer())
```
2. 方便共享变量
因为tf.get_variable() 会检查当前命名空间下是否存在同样name的变量，可以方便共享变量。而tf.Variable 每次都会新建一个变量。

需要注意的是tf.get_variable() 要配合reuse和tf.variable_scope() 使用。
```
with tf.variable_scope("one"):
    a = tf.get_variable("v", [1]) #a.name == "one/v:0"
    
with tf.variable_scope("one"):
    b = tf.get_variable("v", [1]) #创建两个名字一样的变量会报错 ValueError: Variable one/v already exists 
    
with tf.variable_scope("one", reuse = True): #注意reuse的作用。
    c = tf.get_variable("v", [1]) #c.name == "one/v:0" 成功共享，因为设置了reuse

assert a==c #Assertion is true, they refer to the same object.
```
[参考](https://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow)

In [2]:
import tensorflow as tf

#### 1. 在上下文管理器“foo”中创建变量“v”。

In [3]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1], initializer=tf.constant_initializer(1.0))
                        
#with tf.variable_scope("foo"):
   # v = tf.get_variable("v", [1])
    
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
print(v == v1)

#with tf.variable_scope("bar", reuse=True):
   # v = tf.get_variable("v", [1])

True


#### 2. 嵌套上下文管理器中的reuse参数的使用。

In [4]:
with tf.variable_scope("root"):
    print(tf.get_variable_scope().reuse)
    print(tf.get_variable_scope())
    with tf.variable_scope("foo", reuse=True):
        print(tf.get_variable_scope().reuse)
        
        with tf.variable_scope("bar"):
            print(tf.get_variable_scope().reuse)
            
    print(tf.get_variable_scope().reuse)

False
True
True
False


#### 3. 通过variable_scope来管理变量。

In [10]:
print("Leo:",v1.name)
with tf.variable_scope("foo",reuse=True):
    v1 = tf.get_variable("v", [1])
    print(v1.name)

with tf.variable_scope("foo",reuse=True):
    v2 = tf.get_variable("v", [1])
print(v2.name)

with tf.variable_scope("foo"):
    with tf.variable_scope("bar",reuse=tf.AUTO_REUSE):
        v3 = tf.get_variable("v", [1])
        print(v3.name)

v4 = tf.get_variable("v", [1])
print(v4.name)

Leo: foo/v:0
foo/v:0
foo/v:0
foo/bar/v:0
v:0


#### 4. 我们可以通过变量的名称来获取变量。

In [8]:
with tf.variable_scope("", reuse=True):
    v5 = tf.get_variable("foo/bar/v", [1])
    print(v5 == v3)
    v6 = tf.get_variable("v1", [1])     
    print(v6 == v4)

ValueError: Variable foo/bar/v does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?